In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
# ======================== CELL 1: DEPENDENCIES ==========================
import subprocess
import sys

print('🔧 Installing dependencies...')
subprocess.run([sys.executable, "-m", "pip", "uninstall", "-y", "pyarrow"], capture_output=True, check=False)
subprocess.run([sys.executable, "-m", "pip", "install", "-q", "pyarrow==15.0.2", "keybert", "rank-bm25", "faiss-cpu", "sacremoses"], check=True)
print('✅ Dependencies installed. RESTART kernel and run Cell 2.')


In [8]:
# ===================== CELL 2: IMPORTS & SETUP =====================
import warnings
warnings.filterwarnings("ignore")

import re
import json
import time
import pickle
from pathlib import Path
from dataclasses import dataclass
from typing import List, Dict, Tuple

import numpy as np
import torch
import faiss
from datasets import load_dataset
from sentence_transformers import SentenceTransformer
from nltk.tokenize import word_tokenize, sent_tokenize
from rank_bm25 import BM25Okapi

# Device configuration
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"🔧 Using device: {device}")

# ============================================================================
# CONFIGURATION FOR GENERAL MEDICAL DOMAIN
# ============================================================================

@dataclass
class DomainConfig:
    name: str = "general_medical"
    dataset_name: str = "lavita/ChatDoctor-HealthCareMagic-100k"
    dataset_split: str = "train"
    chunk_window: int = 3
    chunk_stride: int = 1
    embed_model: str = "sentence-transformers/all-MiniLM-L6-v2"

config = DomainConfig()

print(f"📋 Building index for: {config.name}")
print(f"📦 Dataset: {config.dataset_name}")


🔧 Using device: cuda
📋 Building index for: general_medical
📦 Dataset: lavita/ChatDoctor-HealthCareMagic-100k


In [9]:
# ===================== CELL 3: DATASET LOADING =====================

def extract_qa_pairs(dataset) -> list:
    """Extract Q&A pairs from ChatDoctor-HealthCareMagic dataset."""
    qa_data = []
    
    for idx, row in enumerate(dataset):
        try:
            # ChatDoctor structure: 'instruction' and 'output'
            if 'instruction' in row and 'output' in row:
                question = str(row['instruction']).strip()
                answer = str(row['output']).strip()
                
                if question and answer and len(answer) > 20:
                    qa_data.append({
                        "question": question,
                        "answer": answer,
                        "source_id": idx
                    })
            
            # Alternative structure: 'input' and 'output'
            elif 'input' in row and 'output' in row:
                question = str(row['input']).strip()
                answer = str(row['output']).strip()
                
                if question and answer and len(answer) > 20:
                    qa_data.append({
                        "question": question,
                        "answer": answer,
                        "source_id": idx
                    })
            
            # Generic Q&A structure
            elif 'question' in row and 'answer' in row:
                question = str(row['question']).strip()
                answer = str(row['answer']).strip()
                
                if question and answer and len(answer) > 20:
                    qa_data.append({
                        "question": question,
                        "answer": answer,
                        "source_id": idx
                    })
                    
        except Exception as e:
            if idx < 3:
                print(f"⚠️ Row {idx} skipped: {e}")
            continue
    
    return qa_data

# Load dataset
print(f"📥 Loading {config.dataset_name}...")
dataset = load_dataset(config.dataset_name, split=config.dataset_split)
print(f"✅ Loaded {len(dataset)} rows")

# Print sample row to verify structure
print(f"\n📋 Sample row structure:")
print(f"Keys: {list(dataset[0].keys())}")
print(f"Sample: {dataset[0]}")

# Extract Q&A pairs
print(f"\n🔍 Extracting Q&A pairs...")
qa_data = extract_qa_pairs(dataset)
print(f"✅ Extracted {len(qa_data)} Q&A pairs")

if len(qa_data) < 1000:
    print(f"⚠️ WARNING: Only {len(qa_data)} pairs extracted from 100k dataset!")
    print(f"Sample extracted pair:")
    if qa_data:
        print(f"Q: {qa_data[0]['question'][:100]}...")
        print(f"A: {qa_data[0]['answer'][:100]}...")


📥 Loading lavita/ChatDoctor-HealthCareMagic-100k...


README.md:   0%|          | 0.00/542 [00:00<?, ?B/s]

data/train-00000-of-00001-5e7cb295b9cff0(…):   0%|          | 0.00/70.5M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/112165 [00:00<?, ? examples/s]

✅ Loaded 112165 rows

📋 Sample row structure:
Keys: ['instruction', 'input', 'output']
Sample: {'instruction': "If you are a doctor, please answer the medical questions based on the patient's description.", 'input': 'I woke up this morning feeling the whole room is spinning when i was sitting down. I went to the bathroom walking unsteadily, as i tried to focus i feel nauseous. I try to vomit but it wont come out.. After taking panadol and sleep for few hours, i still feel the same.. By the way, if i lay down or sit down, my head do not spin, only when i want to move around then i feel the whole world is spinning.. And it is normal stomach discomfort at the same time? Earlier after i relieved myself, the spinning lessen so i am not sure whether its connected or coincidences.. Thank you doc!', 'output': 'Hi, Thank you for posting your query. The most likely cause for your symptoms is benign paroxysmal positional vertigo (BPPV), a type of peripheral vertigo. In this condition, the most co

In [10]:
# ===================== CELL 4: TEXT CHUNKING =====================

def create_chunks(data: List[Dict], window: int = 3, stride: int = 1) -> List[Dict]:
    """Create sentence-level chunks from answers."""
    chunks = []
    
    for item in data:
        text = item.get("answer", "")
        if not text or len(text) < 50:
            continue
        
        sentences = sent_tokenize(text)
        if not sentences:
            continue
        
        # If answer is short, keep as one chunk
        if len(sentences) <= window:
            chunks.append({
                "chunk": " ".join(sentences),
                "source_idx": item.get("source_id", -1),
                "chunk_id": len(chunks)
            })
            continue
        
        # Create overlapping chunks
        for i in range(0, max(1, len(sentences) - window + 1), stride):
            chunks.append({
                "chunk": " ".join(sentences[i:i + window]),
                "source_idx": item.get("source_id", -1),
                "chunk_id": len(chunks)
            })
    
    return chunks

# Create chunks
print(f"🔪 Creating chunks (window={config.chunk_window}, stride={config.chunk_stride})...")
chunks = create_chunks(qa_data, window=config.chunk_window, stride=config.chunk_stride)
print(f"✅ Created {len(chunks)} chunks")

# Sample chunks
print("\n📝 Sample chunks:")
for i, chunk in enumerate(chunks[:3]):
    print(f"{i+1}. {chunk['chunk'][:150]}...")


🔪 Creating chunks (window=3, stride=1)...
✅ Created 710919 chunks

📝 Sample chunks:
1. Hi, Thank you for posting your query. The most likely cause for your symptoms is benign paroxysmal positional vertigo (BPPV), a type of peripheral ver...
2. The most likely cause for your symptoms is benign paroxysmal positional vertigo (BPPV), a type of peripheral vertigo. In this condition, the most comm...
3. In this condition, the most common symptom is dizziness or giddiness, which is made worse with movements. Accompanying nausea and vomiting are common....


In [11]:
# ===================== CELL 5: BUILD FAISS INDEX =====================

# Load embedder
print(f"📦 Loading embedder: {config.embed_model}")
embedder = SentenceTransformer(config.embed_model, device=device)
print(f"✅ Embedder loaded")

# Extract chunk texts
id2doc = [chunk["chunk"] for chunk in chunks]
print(f"📊 Encoding {len(id2doc)} chunks...")

# Create embeddings
embeddings = embedder.encode(
    id2doc,
    normalize_embeddings=True,
    show_progress_bar=True,
    batch_size=32,
    convert_to_numpy=True
).astype('float32')

print(f"✅ Embeddings shape: {embeddings.shape}")

# Build FAISS index
dim = embeddings.shape[1]
index = faiss.IndexFlatIP(dim)
index.add(embeddings)

print(f"✅ FAISS index built: {index.ntotal} vectors, dimension {dim}")


📦 Loading embedder: sentence-transformers/all-MiniLM-L6-v2
✅ Embedder loaded
📊 Encoding 710919 chunks...


Batches:   0%|          | 0/22217 [00:00<?, ?it/s]

✅ Embeddings shape: (710919, 384)
✅ FAISS index built: 710919 vectors, dimension 384


In [12]:
# ===================== CELL 6: BUILD BM25 INDEX =====================

# Tokenize for BM25
print(f"🔨 Building BM25 index...")
bm25_corpus = [word_tokenize(doc.lower()) for doc in id2doc]
bm25 = BM25Okapi(bm25_corpus)
print(f"✅ BM25 index built")


🔨 Building BM25 index...
✅ BM25 index built


In [13]:
# ===================== CELL 7: SAVE INDEXES =====================

# File names
index_file = f"{config.name}_faiss.index"
id2doc_file = f"{config.name}_id2doc.pkl"
metadata_file = f"{config.name}_metadata.json"

# Save FAISS index
print(f"💾 Saving FAISS index to {index_file}...")
faiss.write_index(index, index_file)

# Save id2doc mapping
print(f"💾 Saving id2doc to {id2doc_file}...")
with open(id2doc_file, "wb") as f:
    pickle.dump(id2doc, f)

# Save metadata
metadata = {
    "created_at": time.time(),
    "domain": config.name,
    "dataset": config.dataset_name,
    "n_vectors": int(index.ntotal),
    "embedding_dim": dim,
    "chunk_window": config.chunk_window,
    "chunk_stride": config.chunk_stride,
    "embed_model": config.embed_model
}

print(f"💾 Saving metadata to {metadata_file}...")
with open(metadata_file, "w") as f:
    json.dump(metadata, f, indent=2)

print("\n" + "="*80)
print("✅ INDEX BUILDING COMPLETE!")
print("="*80)
print(f"📁 Files created:")
print(f"  - {index_file}")
print(f"  - {id2doc_file}")
print(f"  - {metadata_file}")
print(f"\n📊 Statistics:")
print(f"  - Total vectors: {index.ntotal}")
print(f"  - Embedding dimension: {dim}")
print(f"  - Source Q&A pairs: {len(qa_data)}")
print(f"  - Chunks created: {len(chunks)}")
print("="*80)


💾 Saving FAISS index to general_medical_faiss.index...
💾 Saving id2doc to general_medical_id2doc.pkl...
💾 Saving metadata to general_medical_metadata.json...

✅ INDEX BUILDING COMPLETE!
📁 Files created:
  - general_medical_faiss.index
  - general_medical_id2doc.pkl
  - general_medical_metadata.json

📊 Statistics:
  - Total vectors: 710919
  - Embedding dimension: 384
  - Source Q&A pairs: 111885
  - Chunks created: 710919


In [14]:
# ===================== CELL 8: VERIFY INDEX (OPTIONAL) =====================

def test_retrieval(query: str, top_k: int = 5):
    """Test the built index with a sample query."""
    print(f"\n🔍 Testing query: '{query}'")
    
    # Embed query
    query_emb = embedder.encode([query], normalize_embeddings=True, convert_to_numpy=True).astype('float32')
    
    # Search FAISS
    D, I = index.search(query_emb, top_k)
    
    print(f"\n📋 Top {top_k} results:")
    for i, (idx, score) in enumerate(zip(I[0], D[0]), 1):
        print(f"\n{i}. Score: {score:.4f}")
        print(f"   {id2doc[idx][:200]}...")

# Test queries
test_queries = [
    "I feel anxious and depressed",
    "How to deal with stress?",
    "I'm having panic attacks"
]

print("\n" + "="*80)
print("🧪 TESTING INDEX")
print("="*80)

for query in test_queries:
    test_retrieval(query, top_k=3)



🧪 TESTING INDEX

🔍 Testing query: 'I feel anxious and depressed'


Batches:   0%|          | 0/1 [00:00<?, ?it/s]


📋 Top 3 results:

1. Score: 0.7162
   You should find an answer. Are you anxious? Are you depressed?...

2. Score: 0.6747
   I suggest you not to worry much. I can see your problem. You know you are depressed, and you have anxiety disorder....

3. Score: 0.6744
   You should understand that you are not depressed. You are dissatisfied, disappointed. Furthermore, you are anxious due to that....

🔍 Testing query: 'How to deal with stress?'


Batches:   0%|          | 0/1 [00:00<?, ?it/s]


📋 Top 3 results:

1. Score: 0.7682
   Consume nutritious food. Lead a stress-free life. I hope my answer will help you....

2. Score: 0.7609
   Don't stress yourself much, take adequate nutritious food on time, as it is the requirement of the body. Focus and concentrate on your career. If you have not studied much, focus on your hobbies and t...

3. Score: 0.7593
   Talk to Someone. Keep a Stress Diary. Take Control....

🔍 Testing query: 'I'm having panic attacks'


Batches:   0%|          | 0/1 [00:00<?, ?it/s]


📋 Top 3 results:

1. Score: 0.7043
   I have gone through your question in detail and I can understand what you are going through. It seems you are having panic attacks, and you certainly need treatment for the same. There are two types o...

2. Score: 0.7021
   If you have panic attacks then you should consult psychiatrist and take treatment accordingly. Hope I have answered your question, if you have doubt then I will be happy to answer. Thanks for using Ch...

3. Score: 0.6996
   Moreover, you have history of panic attacks. In addition, your investigations has come out to be normal. You should consult a psychiatrist for complete evaluation and initiating treatment....
